In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import os
parent_dir = os.path.dirname(os.path.abspath(''))
sys.path.append(parent_dir)

from utils import const

data_path = "../data/binance/BTCUSDT-1m-2024-all-with-indicators.csv"

df = pd.read_csv(data_path)
df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
df = df.set_index("open_time")

In [ ]:
df = df.dropna()

# IC
# factors = ["volume", "count", "quote_volume", "atr", "taker_buy_volume"]
# MAE: 25823.05
# y_predict: 62162.90 y: 70343.3 -> 70345.70
# factors = [
#     "volume",
#     "count",
#     "quote_volume",
#     "atr",
#     "taker_buy_volume",
#     "taker_buy_quote_volume",
#     "obv",
#     "adx",
#     "bb_bbh",
#     "donchian_high",
# ]
# MAE: 126.62
# y_predict: 70489.13 y: 70343.3 -> 70345.70

# IRM
# factors = ["taker_buy_volume", "mfi", "volume", "open", "stoch_d"]
# MAE: 63.06
# y_predict: 70371.22 y: 70343.3 -> 70345.70
# factors = [
#     "taker_buy_volume",
#     "mfi",
#     "volume",
#     "open",
#     "stoch_d",
#     "williams_r",
#     "ema_20",
#     "macd_diff",
#     "bb_bbl",
#     "macd_signal",
# ]
# MAE: 54.38
# y_predict: 70375.95 y: 70343.3 -> 70345.70

# Lasso
# factors = ["taker_buy_quote_volume", "quote_volume"]
# MAE: 31737.10
# y_predict: 61019.25 y: 70343.3 -> 70345.70

# nn
# factors = ["donchian_high", "obv", "mfi", "donchian_low", "high"]
# MAE: 57.12
# y_predict: 70318.09 y: 70343.3 -> 70345.70
# factors = [
#     "donchian_high",
#     "obv",
#     "mfi",
#     "donchian_low",
#     "high",
#     "low",
#     "close",
#     "quote_volume",
#     "count",
#     "volume",
# ]
# MAE: 48.79
# y_predict: 70341.79 y: 70343.3 -> 70345.70


# xgboost
# factors = ["rsi", "bb_bbl", "atr", "donchian_low", "high"]
# MAE: 56.04
# y_predict: 70308.70 y: 70343.3 -> 70345.70
# factors = [
#     "rsi",
#     "bb_bbl",
#     "atr",
#     "donchian_low",
#     "high",
#     "close",
#     "donchian_high",
#     "taker_buy_quote_volume",
#     "macd_signal",
#     "stoch_k",
# ]
# MAE: 48.79
# y_predict: 70343.11 y: 70343.3 -> 70345.70


df["future_close"] = df["close"].shift(-1)
df = df.dropna()

X = df[factors]
y = df["future_close"]
split_time = pd.to_datetime(1730419200000, unit="ms")  # Nov, Dec as test set
X_train = X[X.index < split_time]
X_test = X[X.index >= split_time]
y_train = y[y.index < split_time]
y_test = y[y.index >= split_time]
df_test = df[df.index >= split_time]

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

sample = X_test.iloc[[0]]
predicted = model.predict(sample)
print(
    f"y_predict: {predicted[0]:.2f}",
    f"y: {df_test.iloc[0]['close']} -> {df_test.iloc[0]['future_close']:.2f}",
)

MAE: 48.79
y_predict: 70343.11 y: 70343.3 -> 70345.70
